In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import re
import collections
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import csv
import random

In [2]:
repo_url = 'https://raw.githubusercontent.com/elaynelemos/Lex-POS-Approach/master'
correctSentFile = f'{repo_url}/combined_tagged_lang_selected_entries.csv'

corrSent = pd.read_csv(correctSentFile,header=None,sep="\t")
corrSent = corrSent[2]
tempSent = []
print(len(corrSent))
for sent in corrSent:
    sent = sent.strip()
    sent = re.sub(r'\.{1,}',r'',sent)
    tempSent.append(sent)

bl = list(range(len(corrSent)))

random.shuffle(bl)
corrSent = []
for j in bl :
    corrSent.append(tempSent[j])
    
print(corrSent[0])
print(tempSent[0])

99685
prp i vbd jj cc prp i vbd jj to vb jj nns
jj nn innn on prpd your jj nn


In [3]:
inCorrectSentFile = f'{repo_url}/combined_tagged_specific_error_lang_selected_entries.csv'
IncorrSent = pd.read_csv(inCorrectSentFile,header=None,sep="\t")
IncorrSent = IncorrSent[2]
tempSent = []

for sent in IncorrSent:
    sent = sent.strip()
    sent = re.sub(r'\.{1,}',r'',sent)
    tempSent.append(sent)
    

    
inCorrectSentFile = f'{repo_url}/combined_tagged_general_error_lang_selected_entries.csv'
IncorrSent = pd.read_csv(inCorrectSentFile,header=None,sep="\t")
#IncorrSent = IncorrSent[0]
IncorrSent = IncorrSent[2]

for sent in IncorrSent:
    sent = sent.strip()
    sent = re.sub(r'\.{1,}',r'',sent)
    tempSent.append(sent)
    


bl = list(range(len(tempSent)))
random.shuffle(bl)

IncorrSent = []
for j in bl :
    IncorrSent.append(tempSent[j])
    

print(len(IncorrSent))


92082


In [4]:
corrSent = np.array(corrSent)

zeros = np.zeros((len(corrSent),1),dtype=int) #creating labels of correct sentences
corrSent = np.reshape(corrSent,(len(corrSent),1))
corrSent = np.append(corrSent,zeros,axis=1) #
print(corrSent[2])

['innn before dt this nn ,' '0']


In [5]:
IncorrSent = np.array(IncorrSent)
ones = np.ones((len(IncorrSent),1),dtype=int) #creating labels of incorrect sentences
IncorrSent = np.reshape(IncorrSent,(len(IncorrSent),1))
IncorrSent = np.append(IncorrSent,ones,axis=1)
print(IncorrSent[0])

['rb , nn rb vb vbn' '1']


In [7]:
#partition for specific errors set
'''
trainSet = np.append(corrSent[0:40000,],IncorrSent[0:25000,],axis=0)
trainSet = np.append(trainSet[0:65000,],IncorrSent[10000:25000,],axis=0)
testSet = np.append(corrSent[40000:50000,],IncorrSent[25000:31623,],axis=0)
testSet = np.append(testSet[0:16623,],IncorrSent[11000:14377,],axis=0)
'''
#partition for combined errors set
trainSet = np.append(corrSent[0:48000,],IncorrSent[0:24000,],axis=0)
trainSet = np.append(trainSet[0:72000,],IncorrSent[40000:64000,],axis=0)
testSet = np.append(corrSent[48000:60000,],IncorrSent[24000:30000,],axis=0)
testSet = np.append(testSet[0:18000,],IncorrSent[64000:70000,],axis=0)

'''
#partition for general errors set
trainSet = np.append(corrSent[0:48000,],IncorrSent[0:48000,],axis=0)
testSet = np.append(corrSent[48000:60000,],IncorrSent[48000:60000,],axis=0)
'''
print(len(trainSet))
print(len(testSet))

96000
24000


In [8]:
words_counter = collections.Counter([word  for i in range(len(trainSet)) for word in trainSet[i][0].split()])
#print(words_counter)
print('{} English words.'.format(len([word  for i in range(len(trainSet)) for word in trainSet[i][0].split()])))
print('{} unique English words.'.format(len(words_counter)))
print(words_counter.most_common(10))
print(words_counter)

966682 English words.
1911 unique English words.
[('nn', 103639), ('prp', 81394), ('innn', 57452), ('dt', 55807), ('jj', 52457), ('i', 45054), ('rb', 43325), ('nnp', 37779), ('vbp', 34325), ('vb', 30240)]
Counter({'nn': 103639, 'prp': 81394, 'innn': 57452, 'dt': 55807, 'jj': 52457, 'i': 45054, 'rb': 43325, 'nnp': 37779, 'vbp': 34325, 'vb': 30240, 'vbd': 28789, 'vbz': 26677, 'nns': 24090, 'the': 17743, 'to': 17417, 'prpd': 17138, ',': 16356, 'cc': 14579, 'a': 14435, 'it': 12568, 'vbg': 12436, 'my': 12422, 'vbn': 10864, 'in': 9781, 'md': 9428, 'an': 9072, 'of': 7738, 'for': 6474, 'am': 6122, 'you': 5891, 'this': 5795, 'are': 4785, 'have': 4671, 'that': 4270, 'me': 4130, 'wrb': 3917, 'on': 3773, 'we': 3757, 'at': 3739, 'wp': 3293, 'with': 3181, 'cd': 3056, 'about': 2924, 'pos': 2832, 'L': 2622, 'like': 2592, 'he': 2486, 'ex': 2308, 'A': 2236, 'they': 2136, 'from': 2080, 'she': 1967, 'do': 1958, 'rp': 1943, 'S': 1866, 'want': 1619, 'K': 1594, 'some': 1569, 'because': 1525, 'B': 1524, 'jjr'

In [10]:
def tokenize(vocab,sentences):
    temp = {}
    for w,freq in (vocab.items()):
        if(freq > 2):
            temp[w] = freq
        
                    
    vocab = temp
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 2)} #reserving one for unknown
    vocab_to_int['unk'] = 1
    dataset_int = []
    for sent in sentences:
        #print(sent)
        sentSeq = []
        for word in sent.split():
            if word in vocab_to_int:
                sentSeq.append(vocab_to_int[word])
            else:
                sentSeq.append(vocab_to_int['unk'])
        #print(sentSeq)
        dataset_int.append(sentSeq)
    return dataset_int,vocab_to_int

text_tokenized, text_tokenizer = tokenize(words_counter,trainSet[:,0])
print(text_tokenizer)
print('Count tokens:', len(text_tokenizer))

{'prp': 2, 'i': 3, 'vbd': 4, 'jj': 5, 'cc': 6, 'to': 7, 'vb': 8, 'nns': 9, 'rb': 10, 'nn': 11, 'innn': 12, 'before': 13, 'dt': 14, 'this': 15, ',': 16, 'vbp': 17, 'think': 18, 'am': 19, 'vbg': 20, 'on': 21, 'a': 22, 'P': 23, 'as': 24, 'B': 25, 'jjr': 26, 'have': 27, 'vbn': 28, 'md': 29, 'about': 30, 'prpd': 31, 'my': 32, 'pos': 33, 'it': 34, 'you': 35, 'nnp': 36, 'uh': 37, 'rp': 38, 'for': 39, 'the': 40, 'your': 41, 'because': 42, 'we': 43, 'like': 44, 'with': 45, 'so': 46, 'are': 47, 'vbz': 48, 'than': 49, 'K': 50, 'them': 51, 'that': 52, 'wrb': 53, 'our': 54, 'F': 55, 'after': 56, 'sleep': 57, 'until': 58, 'in': 59, 'me': 60, 'of': 61, 'he': 62, 'she': 63, 'they': 64, 'live': 65, 'ex': 66, 'if': 67, 'want': 68, 'find': 69, 'any': 70, 'cd': 71, 'by': 72, 'L': 73, 'wdt': 74, 'take': 75, 'R': 76, 'hope': 77, '``': 78, 'every': 79, 'some': 80, 'H': 81, 'both': 82, 'N': 83, 'seldom': 84, 'S': 85, 'say': 86, 'make': 87, 'come': 88, 'use': 89, 'need': 90, 'from': 91, 'while': 92, 'please': 

In [12]:
def pad(x, length=None):
       
    if length is None:
        
        length = max([len(sentence) for sentence in x])
        print(length)
        
    return length,pad_sequences(x, maxlen=length, padding='post')
# Pad Tokenized output

seqLen,pad_features = pad(text_tokenized)
print(pad_features[5])

24
[ 7  8 12 24 14 22 25 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [13]:
vocab_size = len(text_tokenizer)+1 # adding 1 for zero value added in padding
print(vocab_size)

731


In [ ]:
#print(pad_features)
#seqLen = 34
def hotcodSeq(pad_features):
    
    temp2 = np.zeros((len(pad_features),seqLen,vocab_size))

    for i_seq in range((len(pad_features))):
        #print("ii "+str(i_seq))
        #print(pad_features[i_seq])
        for j_id in range(len(pad_features[i_seq])):
            if(pad_features[i_seq][j_id]==18):
                temp2[i_seq][j_id][pad_features[i_seq][j_id]] = 1
            else:
                temp2[i_seq][j_id][pad_features[i_seq][j_id]] = 1
        
        
    return temp2
    

features_x = hotcodSeq(pad_features)
print(features_x[5])

In [ ]:
labels = np.array(trainSet[:,1])
labels = labels.astype(int)
print(labels[0])
print(labels.shape)

0
(96000,)


In [ ]:
np.random.seed(14567)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense,Dropout,InputLayer,Embedding
from keras.layers.core import Flatten
def lstmModel(batchSize):
    
    model = Sequential()
    batch_size = batchSize
    model.add(LSTM(50,batch_input_shape=(batch_size,seqLen,vocab_size),return_sequences=True,dropout=0.1,recurrent_dropout=0.1))#output dimensions batch_size,timesteps,20
    model.add(LSTM(50))
    #model.add(Flatten())
    model.add(Dense(64,activation='relu'))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(8,activation='relu'))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
    return model
model = lstmModel(4000)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (4000, 24, 50)            173800    
_________________________________________________________________
lstm_4 (LSTM)                (4000, 50)                20200     
_________________________________________________________________
dense_5 (Dense)              (4000, 64)                3264      
_________________________________________________________________
dense_6 (Dense)              (4000, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (4000, 8)                 264       
_________________________________________________________________
dense_8 (Dense)              (4000, 2)                 18        
Total params: 199,626
Trainable params: 199,626
Non-trainable params: 0
_________________________________________________________________
None

In [ ]:
# fit the model
model.fit(features_x,labels,epochs=500,batch_size=4000,shuffle=True,verbose=1)


Epoch 1/500
52000/96000 [===============>..............] - ETA: 1:48 - loss: 0.6891 - acc: 0.5336

In [ ]:
print(model.metrics_names)

model.evaluate(features_x,labels,batch_size=5000)


['loss', 'acc']
100000/100000 [==============================] - 3s 32us/step


[0.4403184786438942, 0.7725400000810623]

In [ ]:
def text_2_id(tokenizer,sentences):
    dataset_int = []
    for sent in sentences:
        #print(sent)
        sentSeq = []
        for word in sent.split():
            if word in tokenizer:
                sentSeq.append(tokenizer[word])
            else:
                sentSeq.append(tokenizer['unk'])
        #print(sentSeq)
        dataset_int.append(sentSeq)
            
    #for sent in sentences:
        #dataset_int.append([vocab_to_int[w] for word in sent.split() if (word in vocab_to_int):w = word ])
    return dataset_int

testSeq = text_2_id(text_tokenizer,testSet[:,0])
seqLen,testPadSeq = pad(testSeq,seqLen)
#print(testPadSeq)
testLabels = np.array(testSet[:,1])
#print(testPadSeq)
test_features = hotcodSeq(testPadSeq)
print(testLabels.shape)

(30000,)


In [ ]:
print(model.metrics_names)
#model.evaluate(test_features[0:200,],testLabels[0:200,],batch_size=100)
model.evaluate(test_features,testLabels,batch_size=5000)

['loss', 'acc']
30000/30000 [==============================] - 1s 30us/step


[0.5203609764575958, 0.746833344300588]